In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
#from llama_index.embeddings.huggingface_optimum import OptimumEmbedding
from llama_index.core import Settings
from IPython.display import Markdown, display
import chromadb
import torch
from llama_index.llms.huggingface import HuggingFaceLLM


In [2]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

C:\Users\Adel\PycharmProjects\REALREALREAL\venv\Lib\site-packages\torch\nn\modules\module.py:1150: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:30.)
  return t.to(device, dtype if t.is_floating_point() or t.is_complex() else None, non_blocking)


In [3]:
documents = (
    SimpleDirectoryReader("C:/Users/Adel/Desktop/aws-documentation-main/aws-documentation-main/documents/alexa-for-business-administration-guide/doc_source", recursive=True, required_exts=[".md"]).load_data())

In [4]:
from llama_index.core import PromptTemplate
selected_model = "TheBloke/Llama-2-7B-Chat-GPTQ" #pc keeps crashing whenever i use 13b. testing aqlm, aql, gguf and they were slow (vllm speeds it but set-up is a hell on earth) GPTQ is a nice balance. If getting CUDA OUT OF MEMORY, try switching smaller params. WIll be a bit slower but runs on most devices
    

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

In [5]:
llm = HuggingFaceLLM( #play around with the llm parameters here. every number here is either random or from docs - i have not tested new values
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": False}, #need to test different values for this
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    tokenizer_kwargs={"max_length": 3900},
    model_kwargs={"torch_dtype": torch.float16}
)

In [6]:
Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("aws_documentation_test")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [8]:
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("aws_documentation_test")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model
)

In [9]:
query_engine = index.as_query_engine() 
response = query_engine.query("What is alexa for business?")
display(Markdown(f"<b>{response}</b>"))

C:\Users\Adel\PycharmProjects\REALREALREAL\venv\Lib\site-packages\transformers\generation\utils.py:1295: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
C:\Users\Adel\PycharmProjects\REALREALREAL\venv\Lib\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\Adel\PycharmProjects\REALREALREAL\venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:728: UserWarning: 1Torch was not compiled with flash attention. (Triggered intern

<b>Sure, I'd be happy to help! According to the source document, Alexa for Business is a tool that makes it easy for organizations to use Alexa devices, enroll users, and assign skills at scale. It provides tools for building and making available private voice skills for the organization, and also enables voice-enabling products and services for customers.</b>